# ¿Plan Smart o Ultra?  
**Un modelo de clasificación para hacer las recomendaciones correctas**

El objetivo de este proyecto es crear un modelo de machine learning que recomiende el plan telefónico correcto para los clientes de una empresa telefónica llamada Megaline, esto con ayuda de la información recopilada sobre los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos. Para ello, se utilizarán diferentes modelos e hiperparámetros con el fin de investigar cuál es el modelo con la mayor exactitud posible y así asegurar la máxima calidad de las respuestas.

### 1. Pre-procesamiento de Datos

El primer paso es importar todas las librerías necesarias para este proyecto.

In [ ]:
import pandas as pd # Para la manipulación, lectura y análisis de datos
from sklearn.model_selection import train_test_split # Para dividir los datos en distintos conjuntos
from sklearn.metrics import accuracy_score # Para medir la exactitud del modelo de clasificación
from sklearn.tree import DecisionTreeClassifier # Para implementar el modelo de árboles de decisión
from sklearn.ensemble import RandomForestClassifier # Para implementar el modelo de bosque aleatorio
from sklearn.linear_model import LogisticRegression # Para implementar el modelo de regresión logística
from sklearn.dummy import DummyClassifier # Para crear modelos tontos que no aprenden de los datos

Luego, se debe hacer la lectura de los datos y verificar que sean correctos.

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv') # Lectura del dataset
print(df.head(10)) # Imprimir las primeras 10 filas
rows, columns = df.shape # Conocer el número de filas y columnas
print(f'Número de filas: {rows} \nNúmero de columnas: {columns}') 

In [ ]:
df.info() # Con ayuda de info() verificaremos que no haya valores nulos y los tipos sean correctos.

### 2. Segmentación de los datos

Es necesario dividir el dataset en tres partes debido a que no se tiene un conjunto de prueba externo. Para ello, se dividirá en: 
1. **Conjunto de entrenamiento**, para ajustar los parámetros del modelo. 
2. **Conjunto de validación**, para elegir los hiperparámetros y modelos.
3. **Conjunto de prueba**, para su evaluación final.

In [4]:
# El primer paso es definir las variables target y features
target = df['is_ultra']
features = df.drop('is_ultra', axis=1)

In [5]:
# El segundo paso es hacer la división usando train_test_split

 #El 40% de los datos originales se asignará a la variable _temp y el 60% restante a _train
target_train, target_temp, features_train, features_temp = train_test_split(target, features, test_size=0.40, random_state=54321)

# Se tomará el 50% de df_temp para df_test y el otro 50% para df_valid, teniendo el 20% de los datos para cada uno
target_valid, target_test, features_valid, features_test = train_test_split(target_temp, features_temp, test_size=0.50, random_state=54321) 

### 3. Manipulación de los hiperparámetros para encontrar el modelo con mejor desempeño


Es necesario investigar la calidad de diferentes modelos para encontrar el adecuado, para ello se probaran distintos modelos de clasificación (DecisionTreeClassifier, RandomForestClassifier, LogisticRegression) y se manipularán sus hiperparámetros.

**El primer modelo a probar será el de árbol de decisión:**

In [ ]:
# Utilizaremos un bucle for para ajustar max_depth en un rango de 1 a 10
for depth in range(1, 11): 
        model = DecisionTreeClassifier(random_state=54321, max_depth=depth) # Se ajustan los hiperparámetros
        model.fit(features_train, target_train) # Se entrena al modelo usando los datos de entrenamiento
        train_prediction = model.predict(features_train) # Realiza predicciones sobre el mismo conjunto de entrenamiento
        valid_prediction = model.predict(features_valid) # Realiza predicciones sobre el conjunto de validación, esto para elegir la mejor profundidad
        accuracy= accuracy_score(target_valid, valid_prediction) # Compara las predicciones del modelo en el conjunto de validación (valid_prediction) con las verdaderas etiquetas (target_valid)
        print(f"DecisionTreeClassifier max_depth={depth}: Exactitud = {accuracy:}") # Imprime el nivel de profundidad con su exactitud correspondiente

# Ahora se hará una condición if que comparará las puntuaciones de exactitud 
        best_model= None # Inicialmente no se ha determinado cuál es el mejor modelo, por eso se asigna None
        best_accuracy = 0.0 # Guarda la mejor exactitud encontrada hasta el momento, iniciando en 0.0 porque aún no ha probado ningún modelo
        best_params = None # Contendrá los paraámetros del mejor modelo.
    
        # Se comprueba si la exactitud obtenida con el max_depth actual es mejor que la mejor exactitud encontrada hasta el momento (best_accuracy)
        if accuracy > best_accuracy: # Si es mejor, entonces cambia los valores
                best_accuracy = accuracy 
                best_model = model
        
print("Mejor exactitud con Árbol de decisión:", accuracy.max(), "con max_depth:", depth)
print()
print(f"Mejor modelo hasta ahora: {best_model}, con una exactitud de: {best_accuracy}")
         



El árbol de decisión mejora su exactitud a medida que aumenta la profundidad, alcanzando su punto más alto en max_depth=10 con una exactitud aproximada de 0.78. Esto indica que, para este conjunto de datos, un árbol más profundo logra capturar mejor las relaciones entre las características y el objetivo. Por lo tanto, hasta ahora el mejor modelo encontrado es un DecisionTree classifier con max_depth=10.

**Ahora probaremos con un bosque aleatorio, cambiando los hiperparámetros n_estimators y depth**

In [ ]:
for n in range(1, 11): # Utilizaremos un bucle for para ajustar n_estimators en un rango de 1 a 10
    for depth in range(1, 11): 
        model = RandomForestClassifier(random_state=54321, n_estimators=n, max_depth=depth) # Definimos el modelo
        model.fit(features_train, target_train) # Entrenamos al modelo
        valid_prediction = model.predict(features_valid) # Realiza predicciones sobre el conjunto de validación
        accuracy = accuracy_score(target_valid, valid_prediction) # Evaluamos las predicciones anteriores comparándolas con las respuestas correctas
        print(f"RandomForest n_estimators={n}, max_depth={depth}: Exactitud = {accuracy:}")
        
# Ahora haremos una comparación de los porcentajes de exactitud y si este resulta mayor al del modelo anterior, los valores se actualizaran
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_params = {"model": "RandomForest", "n_estimators": n, "max_depth": depth}
print()        
print("Mejor exactitud con Bosque Aleatorio:", accuracy.max())
print()
print(f"Mejor modelo hasta ahora: {best_model}, con una exactitud de: {best_accuracy}")
 



El experimento de hiperparámetros con RandomForestClassifier muestra que el rendimiento mejora con más estimadores (n_estimators) y mayor profundidad del árbol. Tras evaluar distintas combinaciones en el rango de n_estimators= 1 a 10 y max_depth= 1 a 10, se encontró que el mejor resultado se obtiene con max_depth=10 y n_estimators = 8 o 10. Este modelo alcanza una exactitud aproximada de 0.80 y supera al modelo anterior, demostrando que ajustar adecuadamente los hiperparámetros puede mejorar de manera significativa el rendimiento del clasificador.

In [ ]:
# Finalmente probaremos con una prueba de Regresión Logística
model = LogisticRegression(random_state=54321, solver='liblinear') # En este modelo no se cambian los hiperparámetros, sólo se asigna el algoritmo que optimiza el modelo
model.fit(features_train, target_train) # Se entrena el modelo
predictions = model.predict(features_valid) # Se hacen las predicciones con características nuevas
accuracy = accuracy_score(target_valid, predictions) # Se comparan las predicciones con las respuestas correctas
print(f"LogisticRegression, Exactitud = {accuracy}")
print()

# Se comparan las puntuaciones con los modelos anteriores
if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_model = model
    
print(f"Mejor modelo hasta ahora: {best_model}, con una exactitud de: {best_accuracy}")
 



La regresión logística, con una exactitud cercana a 0.68, no supera el desempeño del mejor modelo encontrado hasta ahora. Por lo tanto, el modelo de bosque aleatorio sigue siendo la mejor opción entre las probadas.

### 4. Evaluación final: Comprobación de la calidad del modelo usando el conjunto de prueba

Ya que hemos definido cuál es el mejor modelo, ahora se comprobará su calidad usando un conjunto de prueba.

In [ ]:
test_predictions = best_model.predict(features_test) # Hacemos las predicciones del conjunto de prueba asignándolo a una nueva variable
test_accuracy = accuracy_score(target_test, test_predictions) # Calculamos su puntaje de exactitud comparando las respuestas correctas con las predicciones
print('Exactitud final en el conjunto de prueba:', test_accuracy)

Conclusiones:
El modelo final (que resulto ser RandomForestClassifier(max_depth=10, n_estimators=8, random_state=54321)), alcanzó una exactitud aproximada de 0.82 en el conjunto de prueba. Esto sugiere que el modelo generaliza bien los datos nuevos al mismo tiempo que cumple y supera los objetivos de exactitud planteados.


### 5. Prueba de cordura

Se utiliza una prueba de cordura para asegurar que el modelo mejore incluso si existen resultados triviales o aleatorios. Para ello, se compara el mejor modelo entrenado contra un modelo dummy que no utiliza la información de las características para predecir.

In [ ]:
dummy = DummyClassifier(strategy="most_frequent") # Creamos un modelo dummy que ignora las características y solo predice la clase mayoritaria de un conjunto de datos.
dummy.fit(features_train, target_train) # 'Entrenamos' el modelo
dummy_predictions = dummy.predict(features_test) # Hacemos las predicciones
dummy_accuracy = accuracy_score(target_test, dummy_predictions) # Calculamos la exactitud
print("Exactitud del modelo Dummy:", dummy_accuracy)

La prueba de cordura utilizando un DummyClassifier que siempre predice la clase más frecuente dio una exactitud cercana al 0.72. Dado que el modelo RandomForestClassifier obtuvo un 0.82 en el conjunto de prueba, esto significa que el modelo sí está aprendiendo a partir de las características y supera significativamente una estrategia trivial.

### Conclusión final

La conclusión final del proyecto es que, tras evaluar distintos modelos supervisados de clasificación para machine learning -entre ellos árboles de decisión, bosques aleatorios y regresión logística—, el mejor desempeño se logró con un modelo de bosque aleatorio configurado con n_estimators=8 y max_depth =10. Este modelo alcanzó aproximadamente un 0.82 de exactitud en el conjunto de prueba, superando holgadamente el umbral requerido de 0.75. Además, la prueba de cordura con un modelo Dummy confirmó que el modelo seleccionado aprende relaciones útiles, ya que su exactitud fue superior a la de una predicción trivial.